# Modul 06 - Hamiltonian Simulation

Eine Schulungsserie der Meetup-Gruppe **[Quantum Computing meets Business - Rhineland](https://www.meetup.com/de-DE/Quantum-Computing-meets-Business-Rhineland/)**

(Adapted from [qiskit-textbook](https://github.com/qiskit-community/qiskit-textbook))

In [ ]:
run ./00-Inhalt_Tools.ipynb

## Inhaltsverzeichnis

1. [Ising Modell](#isingmodel)
2. [Schrödinger Gleichung](#schroedinger)
3. [Trotter Approximierung](#trotterization)


## 1. Das Ising Model <a id="isingmodel"></a>

Magneten sind Materialien mit denen wir schon im Kindesalter spielen und die für uns selbstverständlich sind. Aus physikalischer Sicht sind magnetische Materialien jedoch ein komplexes Phänomen, welches erst durch die Entdeckung der Quatenmechanik vollständig erklärt werden konnte. Dahingegen ist das Phänomen des Elektromagnetismus weitgehend durch Schulphysik erklärbar. 

Zur Beschreibung von Materialien mit magnetischen Eigenschaften wurden im Laufe der Zeit verschiedene Modelle entwickelt. Das Einfachste von ihnen ist das sog. Ising-Modell. Das Ising Modell besteht aus mehreren Spins, die miteinander wechselwirken. Zusätzlich gibt es ein externes Magnetfeld, welches die einzelnen Spins beeinflusst. 

Wie alle quantenmechanischen Systeme wird das gesamte Verhalten des Ising-Modells durch seinen Hamilton Operator beschrieben. Dieser Operator beschreibt die Wechselwirkungen der einzelnen Systembestandteile untereinander und mit der Umgebung. Für das Ising Modell ergibt er sich wie folgt: 

$$
\hat{H} = \sum_{<j,k>} J \sigma^z_j \sigma^z_k + \sum_j h \sigma^x_j 
$$

Hierbei ist $J$ die Stärke der Wechselwirkung zwischen den Spins und $h$ die Stärke des externen Magnetfelds. Die einzelnen Spins werden hierbei von den Pauli Operatoren $\sigma^{\alpha}_j$ mit $\alpha \in \lbrace x, y, z\rbrace$ beschrieben. Übertragen auf den Quantencomputer entsprechen sie den Pauli-Gattern $X, Y, Z$. 

## 2. Die Schrödinger Gleichung <a id="schroedinger"></a>

Die Schrödinger Gleichung ist eine der fundamentalsten Gleichungen der Quantenmechanik. Sie beschreibt die zeitliche Entwicklung jedes Quantensystems. Im Allgemeinen hat sie die folgende Form:

$$
\hat{H} \lvert \psi (t) \rangle = -i  \frac{\partial}{\partial t} \lvert \psi (t) \rangle ,
$$

hierbei beschreibt $\lvert \psi \rangle$ den Zustandsvektor des Systems und $\hat{H}$ den bereits oben beschriebenen Hamilton Operator des Systems. Dieser Operator beschreibt das System im Hinblick auf die möglichen Interaktionen mit sich selbst bzw. der Umgebung. Falls dieser Operator nicht von der Zeit abhängig ist, lässt sich die Gleichung formal lösen als:

$$
\lvert \psi (t) \rangle = \exp \left( -i t \hat{H} \right) \lvert \psi (t=0) \rangle .
$$

Kennt man also den Anfangszustand eines Systems und den Hamilton-Operator lässt sich der Zustand des Systems zu jeder beliebigen Zeit durch die obige Gleichung berechnen. In der Praxis steht man jedoch meist vor dem Problem, dass die Gleichung nicht direkt berechenbar ist. An dieser Stelle wollen wir nun den Quantencomputer nutzen, um die Gleichung zu berechnen. Die grundsätzliche Idee ist wie folgt: Zunächst nutzen einen Quantenschaltkreis, um den Zustand zum Zeitpunkt null also den Anfangszustand zu erstellen. Dann erstellen wir einen Schaltkreis, welcher die zeitliche Entwicklung des Systems beschreibt und messen schlussendlich den Endzustand. Aus der Messung des Endzustands können wir diesen Rekonstruieren und haben das System damit simuliert. 

## 3. Die Trotter Approximierung <a id="trotterization"></a>

Während sich der Quantenschaltkreis für den Anfangszustand meist leicht erstellen lässt, ist der Schaltkreis für die zeitliche Entwicklung komplexer. Daher wollen wir zunächst dem mathematischen Objekt den Namen Zeitentwicklungsoperator $U_t$ geben:

$$
U_t = \exp \left( -i t \hat{H} \right).
$$

Da wir den Operator nicht direkt als Schaltkreis implementieren können, müssen wir einige Vereinfachungen vornehmen. Aus der Mathematik wissen wir, dass wir bei Vereinfachungen meistens Fehler machen, deren Größe von der relevanten Skala, hier die Zeit $t$, bestimmt wird. Ein einfacher Trick, um die Zeitschritte zu verkleinern ist den Zeitentwicklungsoperator mehrfach hintereinander für kurze Zeiten anzuwenden:

$$ 
U_t = \prod_{i=1}^{R} U_{t' = t/R}.
$$

Diesen neuen Operator $U_{t'}$ können wir nun weiter vereinfachen. Dazu schreiben wir uns diesen zunächst für das Ising Modell auf:

$$
U_{t'} = \exp\left( -i t' \left(\sum_{j=1}^{N-1} J \sigma^z_j \sigma^z_{j+1} + \sum_{j=1}^{N} h \sigma^x_j \right)\right). 
$$

Als einfache Näherung können wir diesen Operator in zwei Teile teilen, zum einen den Teil mit der Interaktion von Qubits $H_{zz}$ und zum anderen den Teil mit der Wirkung der Magnetfelds $H_x$. Damit ergeben sich dann zwei Operatoren, die wir einzeln genauer anschauen können:

$$
U_{t'} = \exp\left( -i t' \left( H_{zz} + H_{x} \right)\right) \approx \exp\left( -i t' H_{zz} \right) \exp\left( -i t' H_{x} \right).
$$

Nun wollen wir uns zunächst den letzten der beiden Operatoren anschauen. Da der Operator $\sigma^x_j$ nur einzelne Spins bzw. Qubits beeinflusst, lässt sich die Exponentialfunktion einfach auseinander ziehen:

$$ 
\exp\left( -i t' H_{x} \right) = \exp\left( -i t' \sum_{j=1}^{N} h \sigma^x_j \right) = \prod_{j=1}^{N} \exp\left( -i t' h \sigma^x_j \right). 
$$

Aus der Mathematik wissen wir nun, dass es sich bei dem Operator $\exp\left( -i t' h \sigma^x_j \right)$ um eine Drehung um die x-Achse mit dem Winkel $\theta = t' * h$ handelt. Diese können wir sehr einfach als $R_x\left(\theta\right)$-Gatter implementieren. 

Der andere Operator erfordert komplexere Umformungen. Da hier das Produkt zweier Spin bzw. Qubit-Operatoren vorkommt, ist ein einfaches auseinanderziehen eigentlich nicht möglich. Aus mathematischer Sicht liegt dies an der fehlenden Kommutativität der Operatoren $\left[ \sigma^z_1 \sigma^z_2, \sigma^z_2 \sigma^z_3 \right] \neq 0$. Da wir diesen Schritt jedoch trotzdem durchühren wollen, wird ein Fehler in der Größenordnung dieses sog. Kommutators entstehen. In erster Näherung wird dieser Fehler proportional zum Quadrat der Zeitschritte $t'$, der Kopplungskonstaten $J$ und dem Kommutator sein.

$$
\exp\left( -i t' H_{zz} \right) = \exp\left( -i t' \sum_{j=1}^{N-1} J \sigma^z_j \sigma^z_{j+1} \right) \approx \prod_{j=1}^{N-1} \exp\left( -i t' J \sigma^z_j \sigma^z_{j+1} \right)
$$

Falls wir den Fehler in diesem Schritt verkleinern wollen, können wir einerseits die Zeitskala $t'$ verkleinern oder eine Formel höherer Ordnung (siehe auch Suzuki-Trotter Formeln [MISSING REF]) verwenden. Dieser Operator sieht dem $R_z$-Gatter sehr ähnlich. Entsprechend wird es auch als $R_{zz}$-Gatter bezeichnet. Dieses Gatter lässt sich direkt in Qiskit implementieren und wir werden dann sehen, wie es weiter zerlegt werden kann. 

## 4. Implementierung <a id="implementation"></a>

Nach so viel Mathematik und Rechnungen, wollen wir uns nun ein Beispiel anschauen. Dazu wählen wir ein System mit drei Qubits aus und wollen die zeitliche Entwicklung des Anfangszustands $100$ bacg der Zeit $t=\pi$ berrechnen. Dazu müssen wir zunächst die benötigten Module von Qiskit importieren.

In [ ]:
from qiskit import QuantumCircuit

### 4.1. Grundzustand <a id="groundstate"></a>



In [ ]:
groundstate = QuantumCircuit(3)
groundstate.x(0)

## 4. Verweise

[1] 

## Qiskit

In [17]:
import qiskit
qiskit.__qiskit_version__

/Applications/anaconda3/envs/qcmb-training/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'qiskit-terra': '0.16.4',
 'qiskit-aer': '0.7.4',
 'qiskit-ignis': '0.5.2',
 'qiskit-ibmq-provider': '0.11.1',
 'qiskit-aqua': '0.8.2',
 'qiskit': '0.23.5'}

## Copyright

Copyright 2021 Dual Software GmbH and SVA System Vertrieb Alexander GmbH

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.